In [1]:
import cvxpy as cp
import numpy as np

d = 5
K = 2
M = 1

w   = np.random.random(d)
c_0 = np.random.random(d)
c_1 = np.random.random(d)

us = [np.ones(K) for _ in range(M)]

In [6]:
# Construct the problem.
c = cp.Variable(5)
objective = cp.Maximize(w @ c)
constraints = [
    cp.norm(c - c_0, 2) <= 1,
]
prob = cp.Problem(objective, constraints)

In [7]:
# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(c.value)

[0.35920086 0.45874315 0.74245147 0.56634824 0.03915299]


In [3]:
import os 
import pickle 

model_fn = "two_moons_1-2.nf"
cached_fn = os.path.join("/home/yppatel/fusioncp/trained", model_fn)
with open(cached_fn, "rb") as f:
    generative_model = pickle.load(f)
generative_model.to("cpu")        

cached_cpu_fn = os.path.join("/home/yppatel/fusioncp/trained_cpu", model_fn)
with open(cached_cpu_fn, "wb") as f:
    pickle.dump(generative_model, f)